<a href="https://colab.research.google.com/github/SuryaTeja143/Crytpo-Data-Analysis-Dasboard/blob/main/crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
pip install dash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [13]:
import dash
print(dash.__version__)



2.18.2


In [2]:
START_DATE = "2025-01-11" # start date for historical data
RSI_TIME_WINDOW = 7 #number of days


import requests
import pandas as pd
import warnings
import datetime as dt
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas_datareader.data as web
from plotly.subplots import make_subplots
warnings.filterwarnings('ignore')

## URLS and names
urls = ["https://www.cryptodatadownload.com/cdd/Bitfinex_EOSUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_EDOUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_BTCUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_ETHUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_LTCUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_BATUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_OMGUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_DAIUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_ETCUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_ETPUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_NEOUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_REPUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_TRXUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_XLMUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_XMRUSD_d.csv",
        "https://www.cryptodatadownload.com/cdd/Bitfinex_XVGUSD_d.csv",
       ]
crypto_names = ["EOS Coin (EOS)",
                "Eidoo (EDO)",
                "Bitcoin (BTC)",
                "Ethereum (ETH)",
                "Litecoin (LTC)",
                "Basic Attention Token (BAT)",
                "OmiseGO (OMG)",
                "Dai (DAI)",
                "Ethereum Classic (ETC)",
                "Metaverse (ETP)",
                "Neo (NEO)",
                "Augur (REP)",
                "TRON (TRX)",
                "Stellar (XLM)",
                "Monero (XMR)",
                "Verge (XVG)"
               ]

## Data download and loading
def df_loader(urls , start_date = "2021-01-01"):
    filenames = []
    all_df = pd.DataFrame()
    for idx,url in enumerate(urls):
        req = requests.get(url,verify=False)
        url_content = req.content
        filename = url[48:]
        csv_file = open( filename , 'wb')
        csv_file.write(url_content)
        csv_file.close()
        filename = filename[:-9]
        filenames.append(filename)
    for file in filenames:
        df = pd.read_csv(file + "USD_d.csv", header = 1, parse_dates=["date"])
        df = df [df["date"] > start_date]
        df.index = df.date
        df.drop(labels = [df.columns[0],df.columns[1],df.columns[8]] , axis = 1 , inplace = True)
        all_df = pd.concat([all_df,df], ignore_index=False)

    return all_df , filenames
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()
    up_chg = 0 * diff
    down_chg = 0 * diff
    up_chg[diff > 0] = diff[ diff>0 ]
    down_chg[diff < 0] = diff[ diff < 0 ]
    up_chg_avg = up_chg.ewm(com=time_window-1, min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1, min_periods=time_window).mean()
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

all_df , filenames = df_loader(urls , start_date=START_DATE)

crypto_df = []
for file in filenames:
    symbol = file + str("/USD")
    temp_df = pd.DataFrame(all_df[all_df["symbol"] == symbol])
    temp_df.drop(columns= ["symbol"] ,inplace = True)
    temp_df["close_rsi"] = computeRSI(temp_df['close'], time_window=RSI_TIME_WINDOW)
    temp_df["high_rsi"] = 30
    temp_df["low_rsi"] = 70
    exec('%s = temp_df.copy()' % file.lower())
    crypto_df.append(temp_df)

## plot
fig = make_subplots(rows=3,
                    cols=2,
                    shared_xaxes=True,
                    specs=[[{"rowspan": 2}, {"rowspan": 2}], [{"rowspan": 1}, {"rowspan": 1}] , [{},{}]]

                   )
date_buttons = [
{'step': "all", 'label': "All time"},
{'count':  1, 'step': "year", 'stepmode': "backward", 'label': "Last Year"},
{'count':  1, 'step': "year", 'stepmode': "todate", 'label': "Current Year"},
{'count':  1, 'step': "month", 'stepmode': "backward", 'label': "Last 2 Months"},
{'count':  1, 'step': "month", 'stepmode': "todate", 'label': "Current Month"},
{'count':  7, 'step': "day", 'stepmode': "todate", 'label': "Current Week"},
{'count':  4, 'step': "day", 'stepmode': "backward", 'label': "Last 4 days"},
{'count':  1, 'step': "day", 'stepmode': "backward", 'label': "Today"},
               ]
buttons = []
i = 0
j=0
COUNT = 8
vis = [False] * len(crypto_names) * COUNT
for df in crypto_df:
    for k in range(COUNT):
        vis[j+k] = True
    buttons.append({ 'label' : crypto_names[i],
                     'method' : 'update',
                     'args'   : [{'visible' : vis},
                                {'title'  : crypto_names[i] + ' Charts and Indicators'}
                                ] }
                  )
    i+=1
    j+=COUNT
    vis = [False] * len(crypto_names)* COUNT
for df in crypto_df:
    fig.add_trace(
        go.Candlestick(x= df.index,
                       open  = df['open'],
                       high  = df['high'],
                       low   = df['low'],
                       close = df['close'],
                       showlegend =False
                                ),
                  row=1,
                  col=1)
    fig.add_trace(
        go.Bar(x =df.index ,
               y=df["Volume USD"],
               showlegend =False,
               marker_color='aqua'),
        row=3,
        col=1)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['close'],
                   mode='lines',
                  showlegend =False,
                   line=dict(color="red", width=4)),
        row=1,
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index,
                   y=df['low'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False,
                   line=dict(width=2, color='pink', dash='dash')),
        row=1,
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index,
                   y=df['high'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False,
                   line=dict(width=2, color='pink', dash='dash')),
        row=1,
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['close_rsi'],
                   mode='lines',
                  showlegend =False,
                   line=dict(color="aquamarine", width=4)),
        row=3,
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index,
                   y=df['low_rsi'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False,
                   line=dict(width=2, color='aqua', dash='dash')),
        row=3,
        col=2)
    fig.add_trace(
        go.Scatter(x=df.index,
                   y=df['high_rsi'],
                   fill='tonexty',
                   mode='lines',
                   showlegend =False,
                   line=dict(width=2, color='aqua', dash='dash')),
        row=3,
        col=2)


fig.update_xaxes(
        tickfont = dict(size=15, family = 'monospace', color='#B8B8B8'),
        tickmode = 'array',
        ticklen = 6,
        showline = False,
        showgrid = True,
        gridcolor = '#595959',
        ticks = 'outside')
fig.update_layout(
    spikedistance=100,
    xaxis_rangeslider_visible=False,
    hoverdistance=1000)
fig.update_xaxes(
    showspikes=True,
    spikesnap="cursor",
    spikemode="across"
                )
fig.update_yaxes(
    showspikes=True,
    spikesnap='cursor',
    spikemode="across"
                )
fig.update_yaxes(
        tickfont = dict(size=15, family = 'monospace', color='#B8B8B8'),
        tickmode = 'array',
        showline = False,
        ticksuffix = '$',
        showgrid = True,
        gridcolor = '#595959',
        ticks = 'outside')
fig.update_layout(
                  width=1120,
                  height=650,
                  font_family   = 'monospace',
                  xaxis         = dict(rangeselector = dict(buttons = date_buttons)),
                  updatemenus   = [dict(type = 'dropdown',
                                        x = 1.0,
                                        y = 1.108,
                                        showactive = True,
                                        active = 2,
                                        buttons = buttons)],
                  title         = dict(text = '<b>Cryptocurrencies  Dashboard<b>',
                                       font = dict(color = '#FFFFFF' ,size = 22),
                                       x = 0.50),
                  font          = dict(color="blue"),
                  annotations   = [
                                  dict( text = "<b>Choose Cryptocurrency: <b>",
                                        font = dict(size = 20 , color = "#ffffff"),
                                        showarrow=False,
                                        x = 1.02,
                                        y = 1.20,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Candlestick Chart <b>",
                                        font = dict(size = 20,  color = "#ffffff"),
                                        showarrow=False,
                                        x = 0.12,
                                        y = 0.285,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Price Chart<b>",
                                        font = dict(size = 20,  color = "#ffffff"),
                                        showarrow=False,
                                        x = 0.82,
                                        y = 0.285,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Volume Traded<b>",
                                        font = dict(size = 20,  color = "#ffffff"),
                                        showarrow=False,
                                        x = 0.14,
                                        y = -0.53,
                                        xref = 'paper', yref = "paper",
                                        align = "left"),
                                  dict( text = "<b>Relative Strength Index (RSI)<b>",
                                        font = dict(size = 20,  color = "#ffffff"),
                                        showarrow=False,
                                        x = 0.94,
                                        y = -0.53,
                                        xref = 'paper', yref = "paper",
                                        align = "left")
                  ],
                template= "plotly_dark"
                )
for i in range(0,16*COUNT):
    fig.data[i].visible = False
for i in range(COUNT):
    fig.data[i].visible = True
fig.layout["xaxis"]["rangeslider"]["visible"] = False
fig.layout["xaxis2"]["rangeslider"]["visible"] = False
fig.layout["xaxis5"]["rangeslider"]["visible"] = True
fig.layout["xaxis6"]["rangeslider"]["visible"] = True
fig.layout["xaxis5"]["rangeslider"]["borderwidth"] = 4
fig.layout["xaxis6"]["rangeslider"]["borderwidth"] = 4
fig.layout["xaxis5"]["rangeslider"]["bordercolor"] = "aqua"
fig.layout["xaxis6"]["rangeslider"]["bordercolor"] = "aqua"
fig.layout["yaxis6"]["ticksuffix"] = ""
fig.layout["yaxis6"]["range"] = [10,100]
fig.show()

In [15]:
import requests
import pandas as pd
import datetime as dt
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import warnings
import threading

warnings.filterwarnings('ignore')

# Configuration
REFRESH_INTERVAL = 60  # seconds between data refreshes
RSI_TIME_WINDOW = 7  # number of days for RSI calculation

# Cryptocurrency symbols and names
crypto_symbols = [
    "BTCUSD", "ETHUSD", "LTCUSD", "EOSUSD",  "BATUSD",
    "OMGUSD", "DAIUSD", "ETCUSD", "NEOUSD", "REPUSD",
    "TRXUSD", "XLMUSD", "XMRUSD", "XVGUSD"
]

crypto_names = [
    "Bitcoin (BTC)", "Ethereum (ETH)", "Litecoin (LTC)", "EOS Coin (EOS)",
    "Basic Attention Token (BAT)", "OmiseGO (OMG)",
    "Dai (DAI)", "Ethereum Classic (ETC)",
    "Neo (NEO)", "Augur (REP)", "TRON (TRX)", "Stellar (XLM)",
    "Monero (XMR)", "Verge (XVG)"
]

crypto_dict = dict(zip(crypto_symbols, crypto_names))
crypto_data = {}  # Store data for each cryptocurrency

# Function to compute RSI (Relative Strength Index)
def computeRSI(data, time_window):
    diff = data.diff(1).dropna()
    up_chg = 0 * diff
    down_chg = 0 * diff
    up_chg[diff > 0] = diff[diff > 0]
    down_chg[diff < 0] = diff[diff < 0]
    up_chg_avg = up_chg.ewm(com=time_window - 1, min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window - 1, min_periods=time_window).mean()
    rs = abs(up_chg_avg / down_chg_avg)
    rsi = 100 - 100 / (1 + rs)
    return rsi


# Function to fetch data from cryptocurrency API
def fetch_crypto_data(symbol):
    # We'll use CryptoCompare API for real-time data
    url = f"https://min-api.cryptocompare.com/data/v2/histoday"
    params = {
        "fsym": symbol[:3],  # First three characters as crypto symbol
        "tsym": "USD",
        "limit": 90  # Get 90 days of data
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        if data["Response"] == "Success":
            df = pd.DataFrame(data["Data"]["Data"])
            df["time"] = pd.to_datetime(df["time"], unit="s")
            df = df.rename(columns={
                "time": "date",
                "volumefrom": "Volume USD",
                "close": "close",
                "high": "high",
                "low": "low",
                "open": "open"
            })
            df.index = df.date

            # Calculate RSI
            df["close_rsi"] = computeRSI(df['close'], time_window=RSI_TIME_WINDOW)
            df["high_rsi"] = 70
            df["low_rsi"] = 30

            return df
        else:
            print(f"Error fetching data for {symbol}: {data['Message']}")
            return None
    except Exception as e:
        print(f"Exception while fetching {symbol}: {e}")
        return None


# Function to update data for all cryptocurrencies
def update_all_data():
    global crypto_data
    print(f"Updating data at {dt.datetime.now().strftime('%H:%M:%S')}")
    for symbol in crypto_symbols:
        df = fetch_crypto_data(symbol)
        if df is not None:
            crypto_data[symbol] = df
    print("Data update complete.")


# Initial data load
update_all_data()

# Setup the Dash app
app = dash.Dash(__name__,
                meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
                external_stylesheets=[
                    "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.1/css/all.min.css"
                ]
                )
server = app.server

# App layout with modern styling
app.layout = html.Div(
    [
        # Header
        html.H1("Cryptocurrency Dashboard"),

        # Dropdown for selecting cryptocurrency
        dcc.Dropdown(
            id='crypto-dropdown',
            options=[{'label': name, 'value': symbol} for symbol, name in crypto_dict.items()],
            value='BTCUSD'
        ),

        # Graphs
        dcc.Graph(id='candlestick-graph'),
        dcc.Graph(id='volume-graph'),
        dcc.Graph(id='rsi-graph'),

        # Interval for updating data
        dcc.Interval(
            id='data-update',
            interval=REFRESH_INTERVAL * 1000,  # Convert seconds to milliseconds
            n_intervals=0
        )
    ]
)


# Callbacks for updating graphs
@app.callback(
    [Output('candlestick-graph', 'figure'),
     Output('volume-graph', 'figure'),
     Output('rsi-graph', 'figure')],
    [Input('crypto-dropdown', 'value'),
     Input('data-update', 'n_intervals')]
)
def update_graphs(selected_crypto, n):
    df = crypto_data.get(selected_crypto)

    if df is None:
        return {}, {}, {}  # Return empty figures if no data

    # Candlestick chart
    fig_candlestick = go.Figure(data=[go.Candlestick(x=df.index,
                                                        open=df['open'],
                                                        high=df['high'],
                                                        low=df['low'],
                                                        close=df['close'])])
    fig_candlestick.update_layout(title=f"{crypto_dict[selected_crypto]} Candlestick Chart")

    # Volume chart
    fig_volume = go.Figure(data=[go.Bar(x=df.index, y=df['Volume USD'])])
    fig_volume.update_layout(title=f"{crypto_dict[selected_crypto]} Volume Chart")

    # RSI chart
    fig_rsi = go.Figure(data=[go.Scatter(x=df.index, y=df['close_rsi'], mode='lines', name='RSI')])
    fig_rsi.add_trace(go.Scatter(x=df.index, y=[70] * len(df), mode='lines', name='Overbought (70)', line=dict(color='red')))
    fig_rsi.add_trace(go.Scatter(x=df.index, y=[30] * len(df), mode='lines', name='Oversold (30)', line=dict(color='green')))
    fig_rsi.update_layout(title=f"{crypto_dict[selected_crypto]} RSI Chart", yaxis_range=[0, 100])

    return fig_candlestick, fig_volume, fig_rsi


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

# Background thread for updating data
def update_data_periodically():
    while True:
        update_all_data()
        time.sleep(REFRESH_INTERVAL)

threading.Thread(target=update_data_periodically).start()


Updating data at 15:45:47
Data update complete.


<IPython.core.display.Javascript object>

Updating data at 15:45:56
